In this kernel, I'll show where I failed and how to fix it with winning solutions. 

---
## TL;DR
My original model finished public LB 0.9877 with single model, but private LB 0.9035. Obviously I failed to predict unseen labels. I naively assumed public and private were splited randomly, but of course not. My model predict very welll with seen labels, that means it can be still useful to classify seen/unseen labels. After I leaned many things from winner's solutions, I combined these with my models. Here is the result.

Update: After I opened this kernel, Chris @cdeotte advised me how to post process correctly. Thanks, it's really amazing!

| No | model                                | cv    | public | private | rank   |
|----|--------------------------------------|-------|--------|---------|--------|
| 1  | decode 1292 unique labels (original) | 0.999 | 0.9870 | 0.9035  | 1416th |
| 2  | 3 heads for each r, v, c             |       | 0.9891 | 0.9228  | 639th  |
| 3  | 3 models for each r, v, c            |       | 0.9665 | 0.9383  | 56th   |
| 4  | seen: No.1 unseen: No.3              |       | 0.9925 | 0.9552  | 8th    |
| 5  | No.4 with postprocess                |       | 0.9941 | 0.9704  | 2th    |

---

## REFERENCES
I want to say thank you to the author of these references. All of them are great and my code was just conbined these.

### TPU BASELINE
I started from see---'s really great kernels which use TPU as a accelerator. I realized how TPU runs faster than GPU and how easy to use it with keras compared to pytorch xla. Many thanks to see---.
1. https://www.kaggle.com/seesee/1-create-tfrecords
2. https://www.kaggle.com/seesee/2-train
3. https://www.kaggle.com/seesee/3-submit

### TPU DATA AUGMENTATIONS
As see-- himself says, above kernels are plain vanilla one, which means there is a lot of room to improve it. The first thing I wanted to try was data augmentation. But with TPU it's not so easy to apply data augmentation, at least it was for me. But fortunately I found there are many useful kernels to learn how to hundle TPU in [this copmetion](https://www.kaggle.com/c/flower-classification-with-tpus/notebooks), I also want to appreciate these kernel's authors.
1. https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96
2. https://www.kaggle.com/cdeotte/cutmix-and-mixup-on-gpu-tpu
3. https://www.kaggle.com/xiejialun/gridmask-data-augmentation-with-tensorflow

### METRIC LEARNING
After playing around with data augmentation and EfficientNet architectures, my cv score was stacked around 0.986 untils 3~4 days before the deadline. Then I finally found metric learning approach was useful to deal with relatively rare and difficult classes. I mainly refered HumpBack competition's solutions, but I borrowed keras code from the kernel mentioned bellow.

reference solutions
1. https://www.kaggle.com/c/humpback-whale-identification/discussion/82366
2. https://www.kaggle.com/c/humpback-whale-identification/discussion/83885
3. https://www.kaggle.com/c/humpback-whale-identification/discussion/82484

code comes from
1. https://github.com/4uiiurz1/keras-arcface

---

## AFTER FINISHING COMPETITIONS

### SEEN/UNSEEN CLASSIFICATION

Here is the good summary of winner's solution.
https://www.kaggle.com/c/bengaliai-cv19/discussion/136030

As most of winner says, the most important things to achieve good result in private lb was to deal with unseen data. I borrowed the idea to classify seen/unseen by Arcface moudle output from [3rd](https://www.kaggle.com/c/bengaliai-cv19/discussion/135982) or [8th](https://www.kaggle.com/c/bengaliai-cv19/discussion/135990).

### POSTPROCESSING

And also it seems postprocessing to optimize the competition metric macro recall may also work, but somehow I couldn't get good result. WIP.

---

## TRAINING DETAIL
These are not so critical, but I'll summarize what I did.
- initial lr: 0.01 (reduce on plateau with patience=20)
- batch size: 512 (TPU runs way faster if batch size is bigger)
- num_epochs: around 100~150
- backbone: efficientnet b3 (b4~6 also works)
- weights: noisy-student (but almost same as imagenet)
- augment: gridmask + cutmix
- loss: focal loss with gamma=2.0

*The next hedden cell is the change to make the code run with tf2.1.0. Fix minor bug in saving model weight. (It will be fixed in 2.2.0)

In [ ]:
%%writefile /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/keras/saving/hdf5_format.py

# Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
# pylint: disable=protected-access
"""Functions for saving and loading a Keras Model from HDF5 format.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import os

import numpy as np
from six.moves import zip  # pylint: disable=redefined-builtin

from tensorflow.python.framework import ops
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.saving import model_config as model_config_lib
from tensorflow.python.keras.saving import saving_utils
from tensorflow.python.keras.utils import conv_utils
from tensorflow.python.keras.utils.io_utils import ask_to_proceed_with_overwrite
from tensorflow.python.ops import variables as variables_module
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import serialization

# pylint: disable=g-import-not-at-top
try:
  import h5py
  HDF5_OBJECT_HEADER_LIMIT = 64512
except ImportError:
  h5py = None
# pylint: enable=g-import-not-at-top


def save_model_to_hdf5(model, filepath, overwrite=True, include_optimizer=True):
  """Saves a model to a HDF5 file.
  The saved model contains:
      - the model's configuration (topology)
      - the model's weights
      - the model's optimizer's state (if any)
  Thus the saved model can be reinstantiated in
  the exact same state, without any of the code
  used for model definition or training.
  Arguments:
      model: Keras model instance to be saved.
      filepath: One of the following:
          - String, path where to save the model
          - `h5py.File` object where to save the model
      overwrite: Whether we should overwrite any existing
          model at the target location, or instead
          ask the user with a manual prompt.
      include_optimizer: If True, save optimizer's state together.
  Raises:
      ImportError: if h5py is not available.
  """

  if h5py is None:
    raise ImportError('`save_model` requires h5py.')

  # TODO(psv) Add warning when we save models that contain non-serializable
  # entities like metrics added using `add_metric` and losses added using
  # `add_loss.`
  if len(model.weights) != len(model._undeduplicated_weights):
    logging.warning('Found duplicated `Variable`s in Model\'s `weights`. '
                    'This is usually caused by `Variable`s being shared by '
                    'Layers in the Model. These `Variable`s will be treated '
                    'as separate `Variable`s when the Model is restored. To '
                    'avoid this, please save with `save_format="tf"`.')

  if not isinstance(filepath, h5py.File):
    # If file exists and should not be overwritten.
    if not overwrite and os.path.isfile(filepath):
      proceed = ask_to_proceed_with_overwrite(filepath)
      if not proceed:
        return

    f = h5py.File(filepath, mode='w')
    opened_new_file = True
  else:
    f = filepath
    opened_new_file = False

  try:
    model_metadata = saving_utils.model_metadata(model, include_optimizer)
    for k, v in model_metadata.items():
      if isinstance(v, (dict, list, tuple)):
        f.attrs[k] = json.dumps(
            v, default=serialization.get_json_type).encode('utf8')
      else:
        f.attrs[k] = v

    model_weights_group = f.create_group('model_weights')
    model_layers = model.layers
    save_weights_to_hdf5_group(model_weights_group, model_layers)

    # TODO(b/128683857): Add integration tests between tf.keras and external
    # Keras, to avoid breaking TF.js users.
    if (include_optimizer and model.optimizer and
        not isinstance(model.optimizer, optimizers.TFOptimizer)):
      save_optimizer_weights_to_hdf5_group(f, model.optimizer)

    f.flush()
  finally:
    if opened_new_file:
      f.close()


def load_model_from_hdf5(filepath, custom_objects=None, compile=True):  # pylint: disable=redefined-builtin
  """Loads a model saved via `save_model_to_hdf5`.
  Arguments:
      filepath: One of the following:
          - String, path to the saved model
          - `h5py.File` object from which to load the model
      custom_objects: Optional dictionary mapping names
          (strings) to custom classes or functions to be
          considered during deserialization.
      compile: Boolean, whether to compile the model
          after loading.
  Returns:
      A Keras model instance. If an optimizer was found
      as part of the saved model, the model is already
      compiled. Otherwise, the model is uncompiled and
      a warning will be displayed. When `compile` is set
      to False, the compilation is omitted without any
      warning.
  Raises:
      ImportError: if h5py is not available.
      ValueError: In case of an invalid savefile.
  """
  if h5py is None:
    raise ImportError('`load_model` requires h5py.')

  if not custom_objects:
    custom_objects = {}

  opened_new_file = not isinstance(filepath, h5py.File)
  if opened_new_file:
    f = h5py.File(filepath, mode='r')
  else:
    f = filepath

  model = None
  try:
    # instantiate model
    model_config = f.attrs.get('model_config')
    if model_config is None:
      raise ValueError('No model found in config file.')
    model_config = json.loads(model_config.decode('utf-8'))
    model = model_config_lib.model_from_config(model_config,
                                               custom_objects=custom_objects)

    # set weights
    load_weights_from_hdf5_group(f['model_weights'], model.layers)

    if compile:
      # instantiate optimizer
      training_config = f.attrs.get('training_config')
      if training_config is None:
        logging.warning('No training configuration found in the save file, so '
                        'the model was *not* compiled. Compile it manually.')
        return model
      training_config = json.loads(training_config.decode('utf-8'))

      # Compile model.
      model.compile(**saving_utils.compile_args_from_training_config(
          training_config, custom_objects))

      # Set optimizer weights.
      if 'optimizer_weights' in f:
        # Build train function (to get weight updates).
        # Models that aren't graph networks must wait until they are called
        # with data to _make_train_function() and so can't load optimizer
        # weights.
        if model._is_graph_network:  # pylint: disable=protected-access
          if not ops.executing_eagerly_outside_functions():
            model._make_train_function()
          optimizer_weight_values = load_optimizer_weights_from_hdf5_group(f)
          try:
            model.optimizer.set_weights(optimizer_weight_values)
          except ValueError:
            logging.warning('Error in loading the saved optimizer '
                            'state. As a result, your model is '
                            'starting with a freshly initialized '
                            'optimizer.')
        else:
          logging.warning('Sequential models without an `input_shape` '
                          'passed to the first layer cannot reload their '
                          'optimizer state. As a result, your model is'
                          'starting with a freshly initialized optimizer.')

  finally:
    if opened_new_file:
      f.close()
  return model


def preprocess_weights_for_loading(layer,
                                   weights,
                                   original_keras_version=None,
                                   original_backend=None):
  """Preprocess layer weights between different Keras formats.
  Converts layers weights from Keras 1 format to Keras 2 and also weights of
  CuDNN layers in Keras 2.
  Arguments:
      layer: Layer instance.
      weights: List of weights values (Numpy arrays).
      original_keras_version: Keras version for the weights, as a string.
      original_backend: Keras backend the weights were trained with,
          as a string.
  Returns:
      A list of weights values (Numpy arrays).
  """
  def convert_nested_bidirectional(weights):
    """Converts layers nested in `Bidirectional` wrapper.
    This function uses `preprocess_weights_for_loading()` for converting
    layers.
    Arguments:
        weights: List of weights values (Numpy arrays).
    Returns:
        A list of weights values (Numpy arrays).
    """
    num_weights_per_layer = len(weights) // 2
    forward_weights = preprocess_weights_for_loading(
        layer.forward_layer, weights[:num_weights_per_layer],
        original_keras_version, original_backend)
    backward_weights = preprocess_weights_for_loading(
        layer.backward_layer, weights[num_weights_per_layer:],
        original_keras_version, original_backend)
    return forward_weights + backward_weights

  def convert_nested_time_distributed(weights):
    """Converts layers nested in `TimeDistributed` wrapper.
    This function uses `preprocess_weights_for_loading()` for converting nested
    layers.
    Arguments:
        weights: List of weights values (Numpy arrays).
    Returns:
        A list of weights values (Numpy arrays).
    """
    return preprocess_weights_for_loading(
        layer.layer, weights, original_keras_version, original_backend)

  def convert_nested_model(weights):
    """Converts layers nested in `Model` or `Sequential`.
    This function uses `preprocess_weights_for_loading()` for converting nested
    layers.
    Arguments:
        weights: List of weights values (Numpy arrays).
    Returns:
        A list of weights values (Numpy arrays).
    """
    trainable_weights = weights[:len(layer.trainable_weights)]
    non_trainable_weights = weights[len(layer.trainable_weights):]

    new_trainable_weights = []
    new_non_trainable_weights = []

    for sublayer in layer.layers:
      num_trainable_weights = len(sublayer.trainable_weights)
      num_non_trainable_weights = len(sublayer.non_trainable_weights)
      if sublayer.weights:
        preprocessed = preprocess_weights_for_loading(
            layer=sublayer,
            weights=(trainable_weights[:num_trainable_weights] +
                     non_trainable_weights[:num_non_trainable_weights]),
            original_keras_version=original_keras_version,
            original_backend=original_backend)
        new_trainable_weights.extend(preprocessed[:num_trainable_weights])
        new_non_trainable_weights.extend(preprocessed[num_trainable_weights:])

        trainable_weights = trainable_weights[num_trainable_weights:]
        non_trainable_weights = non_trainable_weights[
            num_non_trainable_weights:]

    return new_trainable_weights + new_non_trainable_weights

  # Convert layers nested in Bidirectional/Model/Sequential.
  # Both transformation should be ran for both Keras 1->2 conversion
  # and for conversion of CuDNN layers.
  if layer.__class__.__name__ == 'Bidirectional':
    weights = convert_nested_bidirectional(weights)
  if layer.__class__.__name__ == 'TimeDistributed':
    weights = convert_nested_time_distributed(weights)
  elif layer.__class__.__name__ in ['Model', 'Sequential']:
    weights = convert_nested_model(weights)

  if original_keras_version == '1':
    if layer.__class__.__name__ == 'TimeDistributed':
      weights = preprocess_weights_for_loading(
          layer.layer, weights, original_keras_version, original_backend)

    if layer.__class__.__name__ == 'Conv1D':
      shape = weights[0].shape
      # Handle Keras 1.1 format
      if shape[:2] != (layer.kernel_size[0], 1) or shape[3] != layer.filters:
        # Legacy shape:
        # (filters, input_dim, filter_length, 1)
        assert shape[0] == layer.filters and shape[2:] == (layer.kernel_size[0],
                                                           1)
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))
      weights[0] = weights[0][:, 0, :, :]

    if layer.__class__.__name__ == 'Conv2D':
      if layer.data_format == 'channels_first':
        # old: (filters, stack_size, kernel_rows, kernel_cols)
        # new: (kernel_rows, kernel_cols, stack_size, filters)
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    if layer.__class__.__name__ == 'Conv2DTranspose':
      if layer.data_format == 'channels_last':
        # old: (kernel_rows, kernel_cols, stack_size, filters)
        # new: (kernel_rows, kernel_cols, filters, stack_size)
        weights[0] = np.transpose(weights[0], (0, 1, 3, 2))
      if layer.data_format == 'channels_first':
        # old: (filters, stack_size, kernel_rows, kernel_cols)
        # new: (kernel_rows, kernel_cols, filters, stack_size)
        weights[0] = np.transpose(weights[0], (2, 3, 0, 1))

    if layer.__class__.__name__ == 'Conv3D':
      if layer.data_format == 'channels_first':
        # old: (filters, stack_size, ...)
        # new: (..., stack_size, filters)
        weights[0] = np.transpose(weights[0], (2, 3, 4, 1, 0))

    if layer.__class__.__name__ == 'GRU':
      if len(weights) == 9:
        kernel = np.concatenate([weights[0], weights[3], weights[6]], axis=-1)
        recurrent_kernel = np.concatenate(
            [weights[1], weights[4], weights[7]], axis=-1)
        bias = np.concatenate([weights[2], weights[5], weights[8]], axis=-1)
        weights = [kernel, recurrent_kernel, bias]

    if layer.__class__.__name__ == 'LSTM':
      if len(weights) == 12:
        # old: i, c, f, o
        # new: i, f, c, o
        kernel = np.concatenate(
            [weights[0], weights[6], weights[3], weights[9]], axis=-1)
        recurrent_kernel = np.concatenate(
            [weights[1], weights[7], weights[4], weights[10]], axis=-1)
        bias = np.concatenate(
            [weights[2], weights[8], weights[5], weights[11]], axis=-1)
        weights = [kernel, recurrent_kernel, bias]

    if layer.__class__.__name__ == 'ConvLSTM2D':
      if len(weights) == 12:
        kernel = np.concatenate(
            [weights[0], weights[6], weights[3], weights[9]], axis=-1)
        recurrent_kernel = np.concatenate(
            [weights[1], weights[7], weights[4], weights[10]], axis=-1)
        bias = np.concatenate(
            [weights[2], weights[8], weights[5], weights[11]], axis=-1)
        if layer.data_format == 'channels_first':
          # old: (filters, stack_size, kernel_rows, kernel_cols)
          # new: (kernel_rows, kernel_cols, stack_size, filters)
          kernel = np.transpose(kernel, (2, 3, 1, 0))
          recurrent_kernel = np.transpose(recurrent_kernel, (2, 3, 1, 0))
        weights = [kernel, recurrent_kernel, bias]

  conv_layers = ['Conv1D', 'Conv2D', 'Conv3D', 'Conv2DTranspose', 'ConvLSTM2D']
  if layer.__class__.__name__ in conv_layers:
    if original_backend == 'theano':
      weights[0] = conv_utils.convert_kernel(weights[0])
      if layer.__class__.__name__ == 'ConvLSTM2D':
        weights[1] = conv_utils.convert_kernel(weights[1])
    if K.int_shape(layer.weights[0]) != weights[0].shape:
      weights[0] = np.transpose(weights[0], (3, 2, 0, 1))
      if layer.__class__.__name__ == 'ConvLSTM2D':
        weights[1] = np.transpose(weights[1], (3, 2, 0, 1))

  # convert CuDNN layers
  return _convert_rnn_weights(layer, weights)


def _convert_rnn_weights(layer, weights):
  """Converts weights for RNN layers between native and CuDNN format.
  Input kernels for each gate are transposed and converted between Fortran
  and C layout, recurrent kernels are transposed. For LSTM biases are summed/
  split in half, for GRU biases are reshaped.
  Weights can be converted in both directions between `LSTM` and`CuDNNSLTM`
  and between `CuDNNGRU` and `GRU(reset_after=True)`. Default `GRU` is not
  compatible with `CuDNNGRU`.
  For missing biases in `LSTM`/`GRU` (`use_bias=False`) no conversion is made.
  Arguments:
      layer: Target layer instance.
      weights: List of source weights values (input kernels, recurrent
          kernels, [biases]) (Numpy arrays).
  Returns:
      A list of converted weights values (Numpy arrays).
  Raises:
      ValueError: for incompatible GRU layer/weights or incompatible biases
  """

  def transform_kernels(kernels, func, n_gates):
    """Transforms kernel for each gate separately using given function.
    Arguments:
        kernels: Stacked array of kernels for individual gates.
        func: Function applied to kernel of each gate.
        n_gates: Number of gates (4 for LSTM, 3 for GRU).
    Returns:
        Stacked array of transformed kernels.
    """
    return np.hstack([func(k) for k in np.hsplit(kernels, n_gates)])

  def transpose_input(from_cudnn):
    """Makes a function that transforms input kernels from/to CuDNN format.
    It keeps the shape, but changes between the layout (Fortran/C). Eg.:
    ```
    Keras                 CuDNN
    [[0, 1, 2],  <--->  [[0, 2, 4],
     [3, 4, 5]]          [1, 3, 5]]
    ```
    It can be passed to `transform_kernels()`.
    Arguments:
        from_cudnn: `True` if source weights are in CuDNN format, `False`
            if they're in plain Keras format.
    Returns:
        Function that converts input kernel to the other format.
    """
    order = 'F' if from_cudnn else 'C'

    def transform(kernel):
      return kernel.T.reshape(kernel.shape, order=order)

    return transform

  target_class = layer.__class__.__name__

  # convert the weights between CuDNNLSTM and LSTM
  if target_class in ['LSTM', 'CuDNNLSTM'] and len(weights) == 3:
    # determine if we're loading a CuDNNLSTM layer
    # from the number of bias weights:
    # CuDNNLSTM has (units * 8) weights; while LSTM has (units * 4)
    # if there's no bias weight in the file, skip this conversion
    units = weights[1].shape[0]
    bias_shape = weights[2].shape
    n_gates = 4

    if bias_shape == (2 * units * n_gates,):
      source = 'CuDNNLSTM'
    elif bias_shape == (units * n_gates,):
      source = 'LSTM'
    else:
      raise ValueError('Invalid bias shape: ' + str(bias_shape))

    def convert_lstm_weights(weights, from_cudnn=True):
      """Converts the weights between CuDNNLSTM and LSTM.
      Arguments:
        weights: Original weights.
        from_cudnn: Indicates whether original weights are from CuDNN layer.
      Returns:
        Updated weights compatible with LSTM.
      """

      # Transpose (and reshape) input and recurrent kernels
      kernels = transform_kernels(weights[0], transpose_input(from_cudnn),
                                  n_gates)
      recurrent_kernels = transform_kernels(weights[1], lambda k: k.T, n_gates)
      if from_cudnn:
        # merge input and recurrent biases into a single set
        biases = np.sum(np.split(weights[2], 2, axis=0), axis=0)
      else:
        # Split single set of biases evenly to two sets. The way of
        # splitting doesn't matter as long as the two sets sum is kept.
        biases = np.tile(0.5 * weights[2], 2)
      return [kernels, recurrent_kernels, biases]

    if source != target_class:
      weights = convert_lstm_weights(weights, from_cudnn=source == 'CuDNNLSTM')

  # convert the weights between CuDNNGRU and GRU(reset_after=True)
  if target_class in ['GRU', 'CuDNNGRU'] and len(weights) == 3:
    # We can determine the source of the weights from the shape of the bias.
    # If there is no bias we skip the conversion since
    # CuDNNGRU always has biases.

    units = weights[1].shape[0]
    bias_shape = weights[2].shape
    n_gates = 3

    def convert_gru_weights(weights, from_cudnn=True):
      """Converts the weights between CuDNNGRU and GRU.
      Arguments:
        weights: Original weights.
        from_cudnn: Indicates whether original weights are from CuDNN layer.
      Returns:
        Updated weights compatible with GRU.
      """

      kernels = transform_kernels(weights[0], transpose_input(from_cudnn),
                                  n_gates)
      recurrent_kernels = transform_kernels(weights[1], lambda k: k.T, n_gates)
      biases = np.array(weights[2]).reshape((2, -1) if from_cudnn else -1)
      return [kernels, recurrent_kernels, biases]

    if bias_shape == (2 * units * n_gates,):
      source = 'CuDNNGRU'
    elif bias_shape == (2, units * n_gates):
      source = 'GRU(reset_after=True)'
    elif bias_shape == (units * n_gates,):
      source = 'GRU(reset_after=False)'
    else:
      raise ValueError('Invalid bias shape: ' + str(bias_shape))

    if target_class == 'CuDNNGRU':
      target = 'CuDNNGRU'
    elif layer.reset_after:
      target = 'GRU(reset_after=True)'
    else:
      target = 'GRU(reset_after=False)'

    # only convert between different types
    if source != target:
      types = (source, target)
      if 'GRU(reset_after=False)' in types:
        raise ValueError('%s is not compatible with %s' % types)
      if source == 'CuDNNGRU':
        weights = convert_gru_weights(weights, from_cudnn=True)
      elif source == 'GRU(reset_after=True)':
        weights = convert_gru_weights(weights, from_cudnn=False)

  return weights


def save_optimizer_weights_to_hdf5_group(hdf5_group, optimizer):
  """Saves optimizer weights of a optimizer to a HDF5 group.
  Arguments:
      hdf5_group: HDF5 group.
      optimizer: optimizer instance.
  """

  symbolic_weights = getattr(optimizer, 'weights')
  if symbolic_weights:
    weights_group = hdf5_group.create_group('optimizer_weights')
    weight_names = [str(w.name).encode('utf8') for w in symbolic_weights]
    save_attributes_to_hdf5_group(weights_group, 'weight_names', weight_names)
    weight_values = K.batch_get_value(symbolic_weights)
    for name, val in zip(weight_names, weight_values):
      param_dset = weights_group.create_dataset(
          name, val.shape, dtype=val.dtype)
      if not val.shape:
        # scalar
        param_dset[()] = val
      else:
        param_dset[:] = val


def load_optimizer_weights_from_hdf5_group(hdf5_group):
  """Load optimizer weights from a HDF5 group.
  Arguments:
      hdf5_group: A pointer to a HDF5 group.
  Returns:
      data: List of optimizer weight names.
  """
  weights_group = hdf5_group['optimizer_weights']
  optimizer_weight_names = load_attributes_from_hdf5_group(
      weights_group, 'weight_names')
  return [weights_group[weight_name] for weight_name in optimizer_weight_names]


def save_weights_to_hdf5_group(f, layers):
  """Saves the weights of a list of layers to a HDF5 group.
  Arguments:
      f: HDF5 group.
      layers: List of layer instances.
  """
  from tensorflow.python.keras import __version__ as keras_version  # pylint: disable=g-import-not-at-top

  save_attributes_to_hdf5_group(
      f, 'layer_names', [layer.name.encode('utf8') for layer in layers])
  f.attrs['backend'] = K.backend().encode('utf8')
  f.attrs['keras_version'] = str(keras_version).encode('utf8')

  # Sort model layers by layer name to ensure that group names are strictly
  # growing to avoid prefix issues.
  for layer in sorted(layers, key=lambda x: x.name):
    g = f.create_group(layer.name)
    weights = _legacy_weights(layer)
    weight_values = K.batch_get_value(weights)
    weight_names = [w.name.encode('utf8') for w in weights]
    save_attributes_to_hdf5_group(g, 'weight_names', weight_names)
    for name, val in zip(weight_names, weight_values):
      param_dset = g.create_dataset(name, val.shape, dtype=val.dtype)
      if not val.shape:
        # scalar
        param_dset[()] = val
      else:
        param_dset[:] = val


def load_weights_from_hdf5_group(f, layers):
  """Implements topological (order-based) weight loading.
  Arguments:
      f: A pointer to a HDF5 group.
      layers: a list of target layers.
  Raises:
      ValueError: in case of mismatch between provided layers
          and weights file.
  """
  if 'keras_version' in f.attrs:
    original_keras_version = f.attrs['keras_version'].decode('utf8')
  else:
    original_keras_version = '1'
  if 'backend' in f.attrs:
    original_backend = f.attrs['backend'].decode('utf8')
  else:
    original_backend = None

  filtered_layers = []
  for layer in layers:
    weights = _legacy_weights(layer)
    if weights:
      filtered_layers.append(layer)

  layer_names = load_attributes_from_hdf5_group(f, 'layer_names')
  filtered_layer_names = []
  for name in layer_names:
    g = f[name]
    weight_names = load_attributes_from_hdf5_group(g, 'weight_names')
    if weight_names:
      filtered_layer_names.append(name)
  layer_names = filtered_layer_names
  if len(layer_names) != len(filtered_layers):
    raise ValueError('You are trying to load a weight file '
                     'containing ' + str(len(layer_names)) +
                     ' layers into a model with ' + str(len(filtered_layers)) +
                     ' layers.')

  # We batch weight value assignments in a single backend call
  # which provides a speedup in TensorFlow.
  weight_value_tuples = []
  for k, name in enumerate(layer_names):
    g = f[name]
    weight_names = load_attributes_from_hdf5_group(g, 'weight_names')
    weight_values = [np.asarray(g[weight_name]) for weight_name in weight_names]
    layer = filtered_layers[k]
    symbolic_weights = _legacy_weights(layer)
    weight_values = preprocess_weights_for_loading(
        layer, weight_values, original_keras_version, original_backend)
    if len(weight_values) != len(symbolic_weights):
      raise ValueError('Layer #' + str(k) + ' (named "' + layer.name +
                       '" in the current model) was found to '
                       'correspond to layer ' + name + ' in the save file. '
                       'However the new layer ' + layer.name + ' expects ' +
                       str(len(symbolic_weights)) +
                       ' weights, but the saved weights have ' +
                       str(len(weight_values)) + ' elements.')
    weight_value_tuples += zip(symbolic_weights, weight_values)
  K.batch_set_value(weight_value_tuples)


def load_weights_from_hdf5_group_by_name(
    f, layers, skip_mismatch=False):
  """Implements name-based weight loading.
  (instead of topological weight loading).
  Layers that have no matching name are skipped.
  Arguments:
      f: A pointer to a HDF5 group.
      layers: a list of target layers.
      skip_mismatch: Boolean, whether to skip loading of layers
          where there is a mismatch in the number of weights,
          or a mismatch in the shape of the weights.
  Raises:
      ValueError: in case of mismatch between provided layers
          and weights file and skip_match=False.
  """
  if 'keras_version' in f.attrs:
    original_keras_version = f.attrs['keras_version'].decode('utf8')
  else:
    original_keras_version = '1'
  if 'backend' in f.attrs:
    original_backend = f.attrs['backend'].decode('utf8')
  else:
    original_backend = None

  # New file format.
  layer_names = load_attributes_from_hdf5_group(f, 'layer_names')

  # Reverse index of layer name to list of layers with name.
  index = {}
  for layer in layers:
    if layer.name:
      index.setdefault(layer.name, []).append(layer)

  # We batch weight value assignments in a single backend call
  # which provides a speedup in TensorFlow.
  weight_value_tuples = []
  for k, name in enumerate(layer_names):
    g = f[name]
    weight_names = load_attributes_from_hdf5_group(g, 'weight_names')
    weight_values = [np.asarray(g[weight_name]) for weight_name in weight_names]

    for layer in index.get(name, []):
      symbolic_weights = _legacy_weights(layer)
      weight_values = preprocess_weights_for_loading(
          layer, weight_values, original_keras_version, original_backend)
      if len(weight_values) != len(symbolic_weights):
        if skip_mismatch:
          logging.warning('Skipping loading of weights for '
                          'layer {}'.format(layer.name) + ' due to mismatch '
                          'in number of weights ({} vs {}).'.format(
                              len(symbolic_weights), len(weight_values)))
          continue
        raise ValueError('Layer #' + str(k) + ' (named "' + layer.name +
                         '") expects ' + str(len(symbolic_weights)) +
                         ' weight(s), but the saved weights' + ' have ' +
                         str(len(weight_values)) + ' element(s).')
      # Set values.
      for i in range(len(weight_values)):
        if K.int_shape(symbolic_weights[i]) != weight_values[i].shape:
          if skip_mismatch:
            logging.warning('Skipping loading of weights for '
                            'layer {}'.format(layer.name) + ' due to '
                            'mismatch in shape ({} vs {}).'.format(
                                symbolic_weights[i].shape,
                                weight_values[i].shape))
            continue
          raise ValueError('Layer #' + str(k) +' (named "' + layer.name +
                           '"), weight ' + str(symbolic_weights[i]) +
                           ' has shape {}'.format(K.int_shape(
                               symbolic_weights[i])) +
                           ', but the saved weight has shape ' +
                           str(weight_values[i].shape) + '.')

        else:
          weight_value_tuples.append((symbolic_weights[i], weight_values[i]))
  K.batch_set_value(weight_value_tuples)


def save_attributes_to_hdf5_group(group, name, data):
  """Saves attributes (data) of the specified name into the HDF5 group.
  This method deals with an inherent problem of HDF5 file which is not
  able to store data larger than HDF5_OBJECT_HEADER_LIMIT bytes.
  Arguments:
      group: A pointer to a HDF5 group.
      name: A name of the attributes to save.
      data: Attributes data to store.
  Raises:
    RuntimeError: If any single attribute is too large to be saved.
  """
  # Check that no item in `data` is larger than `HDF5_OBJECT_HEADER_LIMIT`
  # because in that case even chunking the array would not make the saving
  # possible.
  bad_attributes = [x for x in data if len(x) > HDF5_OBJECT_HEADER_LIMIT]

  # Expecting this to never be true.
  if bad_attributes:
    raise RuntimeError('The following attributes cannot be saved to HDF5 '
                       'file because they are larger than %d bytes: %s' %
                       (HDF5_OBJECT_HEADER_LIMIT, ', '.join(bad_attributes)))

  data_npy = np.asarray(data)

  num_chunks = 1
  chunked_data = np.array_split(data_npy, num_chunks)

  # This will never loop forever thanks to the test above.
  while any(x.nbytes > HDF5_OBJECT_HEADER_LIMIT for x in chunked_data):
    num_chunks += 1
    chunked_data = np.array_split(data_npy, num_chunks)

  if num_chunks > 1:
    for chunk_id, chunk_data in enumerate(chunked_data):
      group.attrs['%s%d' % (name, chunk_id)] = chunk_data
  else:
    group.attrs[name] = data


def load_attributes_from_hdf5_group(group, name):
  """Loads attributes of the specified name from the HDF5 group.
  This method deals with an inherent problem
  of HDF5 file which is not able to store
  data larger than HDF5_OBJECT_HEADER_LIMIT bytes.
  Arguments:
      group: A pointer to a HDF5 group.
      name: A name of the attributes to load.
  Returns:
      data: Attributes data.
  """
  if name in group.attrs:
    data = [n.decode('utf8') for n in group.attrs[name]]
  else:
    data = []
    chunk_id = 0
    while '%s%d' % (name, chunk_id) in group.attrs:
      data.extend(
          [n.decode('utf8') for n in group.attrs['%s%d' % (name, chunk_id)]])
      chunk_id += 1
  return data


def _legacy_weights(layer):
  """DO NOT USE.
  For legacy reason, the layer.weights was in the order of
  [self.trainable_weights + self.non_trainable_weights], and this order was
  used for preserving the weights in h5 format. The new order of layer.weights
  are the same as layer.get_weights() which is more intuitive for user. To
  keep supporting the existing saved h5 file, this method should be used to
  save/load weights. In future version, we will delete this method and
  introduce a breaking change for h5 and stay with the new order for weights.
  Args:
    layer: a `tf.keras.Model` or `tf.keras.layers.Layer` instance.
  Returns:
    A list of variables with the order of trainable_weights, followed by
      non_trainable_weights.
  """
  weights = layer.trainable_weights + layer.non_trainable_weights
  if any([not isinstance(w, variables_module.Variable) for w in weights]):
    raise NotImplementedError(
        'Save or restore weights that is not an instance of `tf.Variable` is '
        'not supported in h5, use `save_format=\'tf\'` instead. Got a model '
        'or layer {} with weights {}'.format(layer.__class__.__name__, weights))
  return weights

# INSTALL AND IMPORT LIBRARIES

In [ ]:
!pip install ../input/kaggle-efficientnet-repo/efficientnet-1.0.0-py3-none-any.whl

import os
import numpy as np
import pandas as pd
import argparse
import math
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger)
from tensorflow.keras import regularizers
from tensorflow.keras import layers as L
import efficientnet.tfkeras as efn
from tensorflow.keras.layers import Dense, Lambda, Dropout, Layer, BatchNormalization, Flatten

# UTILS

In [ ]:
def normalize(image):
    # https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/main.py#L325-L326
    # https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/efficientnet_builder.py#L31-L32
    image -= tf.constant([0.485 * 255, 0.456 * 255, 0.406 * 255])  # RGB
    image /= tf.constant([0.229 * 255, 0.224 * 255, 0.225 * 255])  # RGB
    return image

def one_hot(image, label):
    label['root'] = tf.one_hot(label['root'], 168)
    label['vowel'] = tf.one_hot(label['vowel'], 11)
    label['consonant'] = tf.one_hot(label['consonant'], 7)
    label['unique'] = tf.one_hot(label['unique'], 1292)
    label['root2'] = tf.one_hot(label['root2'], 168)
    label['vowel2'] = tf.one_hot(label['vowel2'], 11)
    label['consonant2'] = tf.one_hot(label['consonant2'], 7)
    label['unique2'] = tf.one_hot(label['unique2'], 1292)
    label['root3'] = tf.one_hot(label['root3'], 168)
    label['vowel3'] = tf.one_hot(label['vowel3'], 11)
    label['consonant3'] = tf.one_hot(label['consonant3'], 7)
    label['unique3'] = tf.one_hot(label['unique3'], 1292)
    return image, label

def prepare_metric_learning(image, label, mode='train'):
    if model == 'train':
        return (image, label['root'], label['vowel'], label['consonant'], label['unique']), label
    else:
        return (image, tf.zeros_like(label['root']), tf.zeros_like(label['vowel']), tf.zeros_like(label['consonant']), tf.zeros_like(label['unique'])), label

def get_callbacks(work_dir):
    # model check point
    checkpoint = ModelCheckpoint(work_dir + '/best.h5', 
                                 monitor = 'val_loss', 
                                 verbose = 0, save_best_only=True, 
                                 mode = 'min',
                                 save_weights_only = True)
    csv_logger = CSVLogger(work_dir + '/log.csv')
    early = EarlyStopping(monitor='val_loss', mode='min', patience=20)
    scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10)    
    return [checkpoint, csv_logger, early, scheduler]

def read_tfrecords(example, input_size):
  features = {
      'img': tf.io.FixedLenFeature([], tf.string),
      'image_id': tf.io.FixedLenFeature([], tf.int64),
      'grapheme_root': tf.io.FixedLenFeature([], tf.int64),
      'vowel_diacritic': tf.io.FixedLenFeature([], tf.int64),
      'consonant_diacritic': tf.io.FixedLenFeature([], tf.int64),
      'unique_tuple': tf.io.FixedLenFeature([], tf.int64),
  }
  example = tf.io.parse_single_example(example, features)
  img = tf.image.decode_image(example['img'])
  img = tf.reshape(img, input_size + (1, ))
  img = tf.cast(img, tf.float32)
  # grayscale -> RGB
  img = tf.repeat(img, 3, -1)

  # image_id = tf.cast(example['image_id'], tf.int32)
  grapheme_root = tf.cast(example['grapheme_root'], tf.int32)
  vowel_diacritic = tf.cast(example['vowel_diacritic'], tf.int32)
  consonant_diacritic = tf.cast(example['consonant_diacritic'], tf.int32)
  unique_tuple = tf.cast(example['unique_tuple'], tf.int32)
  #return img, unique_tuple
  return img, {'root': grapheme_root, 'vowel': vowel_diacritic, 'consonant': consonant_diacritic, 'unique': unique_tuple, 'root2': grapheme_root, 'vowel2': vowel_diacritic, 'consonant2': consonant_diacritic, 'unique2': unique_tuple, 'root3': grapheme_root, 'vowel3': vowel_diacritic, 'consonant3': consonant_diacritic, 'unique3': unique_tuple}


# MODEL

In [ ]:
class Generalized_mean_pooling2D(tf.keras.layers.Layer):
    def __init__(self, p=3, epsilon=1e-6, name='', **kwargs):
      super(Generalized_mean_pooling2D, self).__init__(name, **kwargs)
      self.init_p = p
      self.epsilon = epsilon
    
    def build(self, input_shape):
      if isinstance(input_shape, list) or len(input_shape) != 4:
        raise ValueError('`GeM` pooling layer only allow 1 input with 4 dimensions(b, h, w, c)')
      self.build_shape = input_shape
      self.p = self.add_weight(
              name='p',
              shape=[1,],
              initializer=tf.keras.initializers.Constant(value=self.init_p),
              regularizer=None,
              trainable=True,
              dtype=tf.float32
              )
      self.built=True

    def call(self, inputs):
      input_shape = inputs.get_shape()
      if isinstance(inputs, list) or len(input_shape) != 4:
        raise ValueError('`GeM` pooling layer only allow 1 input with 4 dimensions(b, h, w, c)')
      return (tf.reduce_mean(tf.abs(inputs**self.p), axis=[1,2], keepdims=False) + self.epsilon)**(1.0/self.p)


class CosFace(Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.35, regularizer=None, **kwargs):
        super(CosFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(CosFace, self).build(input_shape[0])
        self.W = self.add_weight(shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        target_logits = logits - self.m
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


def get_model(input_size, backbone='efficientnet-b0', weights='imagenet', tta=False):
    print(f'Using backbone {backbone} and weights {weights}')
    x_input = L.Input(shape=input_size, name='imgs', dtype='float32')
    r_label = L.Input(shape=(168,))
    v_label = L.Input(shape=(11,))
    c_label = L.Input(shape=(7,))
    u_label = L.Input(shape=(1292,))

    if backbone.startswith('efficientnet'):
        model_fn = getattr(efn, f'EfficientNetB{backbone[-1]}')

    x = model_fn(input_shape=input_size, weights=weights, include_top=False)(x_input)
    x = Generalized_mean_pooling2D()(x)

    # feature vector
    weight_decay = 1e-4
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    # model architecture is inspired humpback comp's solution.
    # I prepared cosface head and dense head for each outputs.
    
    # root
    x1 = Dense(512, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x1 = BatchNormalization()(x1)
    x1 = tf.nn.l2_normalize(x1, axis=1)
    root = CosFace(168, regularizer=regularizers.l2(weight_decay), name='root')([x1, r_label])
    x1 = Dense(168, use_bias=False)(x1)
    root2 = Lambda(lambda x: K.softmax(x), name='root2')(x1)

    # vowel
    x2 = Dense(512, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x2 = BatchNormalization()(x2)
    x2 = tf.nn.l2_normalize(x2, axis=1)
    vowel = CosFace(11, regularizer=regularizers.l2(weight_decay), name='vowel')([x2, v_label])
    x2 = Dense(11, use_bias=False)(x2)
    vowel2 = Lambda(lambda x: K.softmax(x), name='vowel2')(x2)

    # consonant
    x3 = Dense(512, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x3 = BatchNormalization()(x3)
    x3 = tf.nn.l2_normalize(x3, axis=1)
    consonant = CosFace(7, regularizer=regularizers.l2(weight_decay), name='consonant')([x3, c_label])
    x3 = Dense(7, use_bias=False)(x3)
    consonant2 = Lambda(lambda x: K.softmax(x), name='consonant2')(x3)

    # unique
    x4 = Dense(1024, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x4 = BatchNormalization()(x4)
    x4 = tf.nn.l2_normalize(x4, axis=1)
    unique = CosFace(1292, regularizer=regularizers.l2(weight_decay), name='unique')([x4, u_label])
    x4 = Dense(1292, use_bias=False)(x4)
    unique2 = Lambda(lambda x: K.softmax(x), name='unique2')(x4)

    # I thought it may useful to know what is other heads' prediction, so I concat all logits and predict each output again.
    # But it turned out this head ddin't perform better than other branch. You may remove it.

    # concat all logits
    xx = tf.concat([x1, x2, x3, x4], axis=1)
    xx = tf.keras.activations.relu(xx)

    # Wow, I just realized these 4 dense layers exist when I refactored the codes....lol
    # These are not supposed to be here, but my final model weight use them, so I don't remove it...
    xx1 = Dense(168)(xx)
    xx2 = Dense(11)(xx)
    xx3 = Dense(7)(xx)
    xx4 = Dense(1292)(xx)

    xx1 = Dense(168, use_bias=False)(xx1)
    root3 = Lambda(lambda x: K.softmax(x), name='root3')(xx1)

    xx2 = Dense(11, use_bias=False)(xx2)
    vowel3 = Lambda(lambda x: K.softmax(x), name='vowel3')(xx2)

    xx3 = Dense(7, use_bias=False)(xx3)
    consonant3 = Lambda(lambda x: K.softmax(x), name='consonant3')(xx3)

    xx4 = Dense(1292, use_bias=False)(xx4)
    unique3 = Lambda(lambda x: K.softmax(x), name='unique3')(xx4)

    # model
    model = tf.keras.Model(
        inputs = [x_input,r_label,v_label,c_label,u_label],
        outputs = [root, vowel, consonant, unique, root2, vowel2, consonant2, unique2, root3, vowel3, consonant3, unique3]
    )

    return model

# Additionaly, I trained 3 models for unseen labels for each root, vowel and consonant prediction.
# In this kernel, you can train only above multi-head model. To train below 3 models, you need to modify the codes a little bit.

def get_r_model(input_size, backbone='efficientnet-b0', weights='imagenet', tta=False):
    print(f'Using backbone {backbone} and weights {weights}')
    x_input = L.Input(shape=input_size, name='imgs', dtype='float32')
    r_label = L.Input(shape=(168,))

    if backbone.startswith('efficientnet'):
        model_fn = getattr(efn, f'EfficientNetB{backbone[-1]}')

    x = model_fn(input_shape=input_size, weights=weights, include_top=False)(x_input)
    x = Generalized_mean_pooling2D()(x)

    # feature vector
    weight_decay = 1e-4
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    # root
    x1 = Dense(512, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x1 = BatchNormalization()(x1)
    x1 = tf.nn.l2_normalize(x1, axis=1)
    root = CosFace(168, regularizer=regularizers.l2(weight_decay), name='root')([x1, r_label])
    x1 = Dense(168, use_bias=False)(x1)
    root2 = Lambda(lambda x: K.softmax(x), name='root2')(x1)

    # model
    model = tf.keras.Model(
        inputs = [x_input,r_label],
        outputs = [root, root2]
    )

    return model

def get_v_model(input_size, backbone='efficientnet-b0', weights='imagenet', tta=False):
    print(f'Using backbone {backbone} and weights {weights}')
    x_input = L.Input(shape=input_size, name='imgs', dtype='float32')
    v_label = L.Input(shape=(11,))

    if backbone.startswith('efficientnet'):
        model_fn = getattr(efn, f'EfficientNetB{backbone[-1]}')

    x = model_fn(input_shape=input_size, weights=weights, include_top=False)(x_input)
    x = Generalized_mean_pooling2D()(x)

    # feature vector
    weight_decay = 1e-4
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    # vowel
    x2 = Dense(512, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x2 = BatchNormalization()(x2)
    x2 = tf.nn.l2_normalize(x2, axis=1)
    vowel = CosFace(11, regularizer=regularizers.l2(weight_decay), name='vowel')([x2, v_label])
    x2 = Dense(11, use_bias=False)(x2)
    vowel2 = Lambda(lambda x: K.softmax(x), name='vowel2')(x2)

    # model
    model = tf.keras.Model(
        inputs = [x_input,v_label],
        outputs = [vowel, vowel2]
    )

    return model

def get_c_model(input_size, backbone='efficientnet-b0', weights='imagenet', tta=False):
    print(f'Using backbone {backbone} and weights {weights}')
    x_input = L.Input(shape=input_size, name='imgs', dtype='float32')
    c_label = L.Input(shape=(7,))

    if backbone.startswith('efficientnet'):
        model_fn = getattr(efn, f'EfficientNetB{backbone[-1]}')

    x = model_fn(input_shape=input_size, weights=weights, include_top=False)(x_input)
    x = Generalized_mean_pooling2D()(x)

    # feature vector
    weight_decay = 1e-4
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    # consonant
    x3 = Dense(512, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x3 = BatchNormalization()(x3)
    x3 = tf.nn.l2_normalize(x3, axis=1)
    consonant = CosFace(7, regularizer=regularizers.l2(weight_decay), name='consonant')([x3, c_label])
    x3 = Dense(7, use_bias=False)(x3)
    consonant2 = Lambda(lambda x: K.softmax(x), name='consonant2')(x3)

    # model
    model = tf.keras.Model(
        inputs = [x_input,c_label],
        outputs = [consonant, consonant2]
    )

    return model


# LOSS

In [ ]:
def categorical_focal_loss(num_classes, gamma=2., alpha=.25, smooth_alpha=0.05):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """
        if smooth_alpha > 0:
            y_true = y_true * (1 - smooth_alpha) + smooth_alpha / num_classes

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return K.sum(loss, axis=1)

    return categorical_focal_loss_fixed  

# AUGMENTATIONS

In [ ]:
def cutmix(images, labels, batch_size, image_size):
    
    DIM = image_size[0]
    
    # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
    # This is a tensor containing 0 or 1 -- 0: no cutmix.
    # shape = [batch_size]
    #do_cutmix = tf.cast(tf.random.uniform([batch_size], 0, 1) <= 1.0, tf.int32)
    
    # Choose random images in the batch for cutmix
    # shape = [batch_size]
    new_image_indices = tf.cast(tf.random.uniform([batch_size], 0, batch_size), tf.int32)
    
    # Choose random location in the original image to put the new images
    # shape = [batch_size]
    new_x = tf.cast(tf.random.uniform([batch_size], 0, DIM), tf.int32)
    new_y = tf.cast(tf.random.uniform([batch_size], 0, DIM), tf.int32)
    
    # Random width for new images, shape = [batch_size]
    b = tf.random.uniform([batch_size], 0, 1) # this is beta dist with alpha=1.0
    new_width = tf.cast(DIM * tf.math.sqrt(1-b), tf.int32) #* do_cutmix
    
    # shape = [batch_size]
    new_y0 = tf.math.maximum(0, new_y - new_width // 2)
    new_y1 = tf.math.minimum(DIM, new_y + new_width // 2)
    new_x0 = tf.math.maximum(0, new_x - new_width // 2)
    new_x1 = tf.math.minimum(DIM, new_x + new_width // 2)
    
    # shape = [batch_size, DIM]
    target = tf.broadcast_to(tf.range(DIM), shape=(batch_size, DIM))
    
    # shape = [batch_size, DIM]
    mask_y = tf.math.logical_and(new_y0[:, tf.newaxis] <= target, target <= new_y1[:, tf.newaxis])
    
    # shape = [batch_size, DIM]
    mask_x = tf.math.logical_and(new_x0[:, tf.newaxis] <= target, target <= new_x1[:, tf.newaxis])    
    
    # shape = [batch_size, DIM, DIM]
    mask = tf.cast(tf.math.logical_and(mask_y[:, :, tf.newaxis], mask_x[:, tf.newaxis, :]), tf.float32)

    # All components are of shape [batch_size, DIM, DIM, 3]
    new_images =  images * tf.broadcast_to(1 - mask[:, :, :, tf.newaxis], [batch_size, DIM, DIM, 3]) + \
                    tf.gather(images, new_image_indices) * tf.broadcast_to(mask[:, :, :, tf.newaxis], [batch_size, DIM, DIM, 3])

    a = tf.cast(new_width ** 2 / DIM ** 2, tf.float32)    
        
    #new_labels =  (1-a)[:, tf.newaxis] * labels + a[:, tf.newaxis] * tf.gather(labels, new_image_indices)        
    for c in ['root', 'vowel', 'consonant', 'unique', 'root2', 'vowel2', 'consonant2', 'unique2', 'root3', 'vowel3', 'consonant3', 'unique3']:
        y1, y2 = labels[c], tf.gather(labels[c], new_image_indices)
        labels[c] = y1 * (1-a)[:, tf.newaxis] + y2 * a[:, tf.newaxis]

    return new_images, labels

def transform(image, inv_mat, image_shape):

    h, w, c = image_shape
    cx, cy = w//2, h//2

    new_xs = tf.repeat( tf.range(-cx, cx, 1), h)
    new_ys = tf.tile( tf.range(-cy, cy, 1), [w])
    new_zs = tf.ones([h*w], dtype=tf.int32)

    old_coords = tf.matmul(inv_mat, tf.cast(tf.stack([new_xs, new_ys, new_zs]), tf.float32))
    old_coords_x, old_coords_y = tf.round(old_coords[0, :] + w//2), tf.round(old_coords[1, :] + h//2)

    clip_mask_x = tf.logical_or(old_coords_x<0, old_coords_x>w-1)
    clip_mask_y = tf.logical_or(old_coords_y<0, old_coords_y>h-1)
    clip_mask = tf.logical_or(clip_mask_x, clip_mask_y)

    old_coords_x = tf.boolean_mask(old_coords_x, tf.logical_not(clip_mask))
    old_coords_y = tf.boolean_mask(old_coords_y, tf.logical_not(clip_mask))
    new_coords_x = tf.boolean_mask(new_xs+cx, tf.logical_not(clip_mask))
    new_coords_y = tf.boolean_mask(new_ys+cy, tf.logical_not(clip_mask))

    old_coords = tf.cast(tf.stack([old_coords_y, old_coords_x]), tf.int32)
    new_coords = tf.cast(tf.stack([new_coords_y, new_coords_x]), tf.int64)
    rotated_image_values = tf.gather_nd(image, tf.transpose(old_coords))
    rotated_image_channel = list()
    for i in range(c):
        vals = rotated_image_values[:,i]
        sparse_channel = tf.SparseTensor(tf.transpose(new_coords), vals, [h, w])
        rotated_image_channel.append(tf.sparse.to_dense(sparse_channel, default_value=0, validate_indices=False))

    return tf.transpose(tf.stack(rotated_image_channel), [1,2,0])

def random_rotate(image, angle, image_shape):

    def get_rotation_mat_inv(angle):
          #transform to radian
        angle = math.pi * angle / 180

        cos_val = tf.math.cos(angle)
        sin_val = tf.math.sin(angle)
        one = tf.constant([1], tf.float32)
        zero = tf.constant([0], tf.float32)

        rot_mat_inv = tf.concat([cos_val, sin_val, zero,
                                     -sin_val, cos_val, zero,
                                     zero, zero, one], axis=0)
        rot_mat_inv = tf.reshape(rot_mat_inv, [3,3])

        return rot_mat_inv
    angle = float(angle) * tf.random.normal([1],dtype='float32')
    rot_mat_inv = get_rotation_mat_inv(angle)
    return transform(image, rot_mat_inv, image_shape)


def GridMask(image_height, image_width, d1, d2, rotate_angle=1, ratio=0.5):

    h, w = image_height, image_width
    hh = int(np.ceil(np.sqrt(h*h+w*w)))
    hh = hh+1 if hh%2==1 else hh
    d = tf.random.uniform(shape=[], minval=d1, maxval=d2, dtype=tf.int32)
    l = tf.cast(tf.cast(d,tf.float32)*ratio+0.5, tf.int32)

    st_h = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)
    st_w = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)

    y_ranges = tf.range(-1 * d + st_h, -1 * d + st_h + l)
    x_ranges = tf.range(-1 * d + st_w, -1 * d + st_w + l)

    for i in range(0, hh//d+1):
        s1 = i * d + st_h
        s2 = i * d + st_w
        y_ranges = tf.concat([y_ranges, tf.range(s1,s1+l)], axis=0)
        x_ranges = tf.concat([x_ranges, tf.range(s2,s2+l)], axis=0)

    x_clip_mask = tf.logical_or(x_ranges <0 , x_ranges > hh-1)
    y_clip_mask = tf.logical_or(y_ranges <0 , y_ranges > hh-1)
    clip_mask = tf.logical_or(x_clip_mask, y_clip_mask)

    x_ranges = tf.boolean_mask(x_ranges, tf.logical_not(clip_mask))
    y_ranges = tf.boolean_mask(y_ranges, tf.logical_not(clip_mask))

    hh_ranges = tf.tile(tf.range(0,hh), [tf.cast(tf.reduce_sum(tf.ones_like(x_ranges)), tf.int32)])
    x_ranges = tf.repeat(x_ranges, hh)
    y_ranges = tf.repeat(y_ranges, hh)

    y_hh_indices = tf.transpose(tf.stack([y_ranges, hh_ranges]))
    x_hh_indices = tf.transpose(tf.stack([hh_ranges, x_ranges]))

    y_mask_sparse = tf.SparseTensor(tf.cast(y_hh_indices, tf.int64),  tf.zeros_like(y_ranges), [hh, hh])
    y_mask = tf.sparse.to_dense(y_mask_sparse, 1, False)

    x_mask_sparse = tf.SparseTensor(tf.cast(x_hh_indices, tf.int64), tf.zeros_like(x_ranges), [hh, hh])
    x_mask = tf.sparse.to_dense(x_mask_sparse, 1, False)

    mask = tf.expand_dims( tf.clip_by_value(x_mask + y_mask, 0, 1), axis=-1)

    mask = random_rotate(mask, rotate_angle, [hh, hh, 1])
    mask = tf.image.crop_to_bounding_box(mask, (hh-h)//2, (hh-w)//2, image_height, image_width)

    return mask

def apply_grid_mask(image, image_shape):
    AugParams = {
        'd1' : 10,
        'd2': 100,
        'rotate' : 15,
        'ratio' : 0.5
    }
    mask = GridMask(image_shape[0],
                    image_shape[1],
                    AugParams['d1'],
                    AugParams['d2'],
                    AugParams['rotate'],
                    #AugParams['ratio'],
                    tf.random.uniform(shape=[], minval=0.3, maxval=0.6)
                    )
    
    if image_shape[-1] == 3:
        mask = tf.concat([mask, mask, mask], axis=-1)

    return image * tf.cast(mask, tf.float32)

def augmentation(image, label, input_size):
    image = apply_grid_mask(image, (*input_size,3))
    return image, label

# MAIN

you can train the models by changing epochs and turning on TPU accelerator.

In [ ]:
EXP_NAME = 'efnet-b6_cosface_gridmask_cutmix'

parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=123)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--input_size', type=str, default='224,224')
parser.add_argument('--batch_size', type=int, default=512)
parser.add_argument('--epochs', type=int, default=0)
parser.add_argument('--backbone', type=str, default='efficientnet-b3')
parser.add_argument('--weights', type=str, default='noisy-student')
parser.add_argument('--root_dir', type=str, default='./')
parser.add_argument('--resume_from', type=str, default=None)
args, _ = parser.parse_known_args([
    '--lr' '0.0001',
    '--batch_size', '32',
    '--aug', 'gridmask',
    '--mix', 'cutmix',
])

work_dir = args.root_dir + EXP_NAME
os.makedirs(work_dir, exist_ok=True)

args.input_size = tuple(int(x) for x in args.input_size.split(','))
np.random.seed(args.seed)
tf.random.set_seed(args.seed)

if args.epochs > 0:
    try:
      tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
      print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    except ValueError:
      raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)


    with strategy.scope():
        model = get_model(input_size=args.input_size + (3, ), backbone=args.backbone,
        weights=args.weights)

        model.compile(optimizer = Adam(lr = args.lr),
                        loss = {'root': categorical_focal_loss(168),
                                'vowel': categorical_focal_loss(11),
                                'consonant': categorical_focal_loss(7),
                                'unique': categorical_focal_loss(1292),
                                'root2': categorical_focal_loss(168),
                                'vowel2': categorical_focal_loss(11),
                                'consonant2': categorical_focal_loss(7),
                                'unique2': categorical_focal_loss(1292),
                                'root3': categorical_focal_loss(168),
                                'vowel3': categorical_focal_loss(11),
                                'consonant3': categorical_focal_loss(7),
                                'unique3': categorical_focal_loss(1292)},
                        loss_weights = {'root': 0.25,        
                                        'vowel': 0.25,
                                        'consonant': 0.25,
                                        'unique': 0.25,
                                        'root2': 0.25,        
                                        'vowel2': 0.25,
                                        'consonant2': 0.25,
                                        'unique2': 0.25,
                                        'root3': 0.25,        
                                        'vowel3': 0.25,
                                        'consonant3': 0.25,
                                        'unique3': 0.25},
                        metrics = { 'root': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'vowel': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'consonant': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'unique': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'root2': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'vowel2': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'consonant2': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'unique2': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'root3': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'vowel3': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'consonant3': ['categorical_accuracy', tf.keras.metrics.Recall()],
                                    'unique3': ['categorical_accuracy', tf.keras.metrics.Recall()]
                                                }
                        )

    from kaggle_datasets import KaggleDatasets
    ds_path = KaggleDatasets().get_gcs_path('tfrecords-224')
    train_fns = tf.io.gfile.glob(os.path.join(ds_path, 'train*.tfrec'))
    val_fns = tf.io.gfile.glob(os.path.join(ds_path, 'val*.tfrec'))

    # get train dataset with augmentations
    AUTO = tf.data.experimental.AUTOTUNE
    train_ds = tf.data.TFRecordDataset(train_fns)
    train_ds = train_ds.map(lambda e: read_tfrecords(e, args.input_size))
    train_ds = train_ds.map(lambda a, b: augmentation(a, b, args.input_size), num_parallel_calls=AUTO)
    train_ds = train_ds.repeat().batch(args.batch_size)
    train_ds = train_ds.map(one_hot)
    train_ds = train_ds.map(lambda a, b: cutmix(a, b, args.batch_size, args.input_size))
    train_ds = train_ds.map(lambda a, b: prepare_metric_learning(a, b, 'train'))

    # get valid dataset
    val_ds = tf.data.TFRecordDataset(val_fns)
    val_ds = val_ds.map(lambda e: read_tfrecords(e, args.input_size))
    val_ds = val_ds.batch(args.batch_size)
    val_ds = val_ds.map(one_hot)
    val_ds = val_ds.map(lambda a, b: prepare_metric_learning(a, b, 'valid'))

    # train
    print(EXP_NAME)
    if args.resume_from:
        checkpoint_path = args.resume_from
        print('load model from ', checkpoint_path)
        model.load_weights(checkpoint_path)

    num_train_samples = sum(int(fn.split('_')[2]) for fn in train_fns)
    steps_per_epoch = num_train_samples // args.batch_size
    print(f'Training on {num_train_samples} samples. Each epochs requires {steps_per_epoch} steps')
    h = model.fit(train_ds, steps_per_epoch=steps_per_epoch, epochs=args.epochs, verbose=1,
        validation_data=val_ds, callbacks=get_callbacks(work_dir))

# INFERENCE

1. Predict with multi-head model.
2. Classify seen/unseen by maximum value of cosface head's output.
3. For seen data, use multi-heads model r, c, v outputs.
4. For unseen data, use separate 3 models for each r, c, v.

In [ ]:
import cv2
import numpy as np
import os
from tqdm.notebook import tqdm
import gc

def normalize_image(img, org_width, org_height, new_width, new_height):
  # Invert
  img = 255 - img
  # Normalize
  img = (img * (255.0 / img.max())).astype(np.uint8)
  # Reshape
  img = img.reshape(org_height, org_width)
  image_resized = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
  return image_resized

        
def predict_batch(img_batch, model, all_preds, unseen_th=0.5):
    img_batch = np.float32(img_batch)
    bs = img_batch.shape[0]
    r,v,c,u = np.zeros((bs,168)), np.zeros((bs,11)), np.zeros((bs,7)), np.zeros((bs,1292))
    # deal with single image
    if img_batch.ndim != 4:
        img_batch = np.expand_dims(img_batch, 0)

    y_pred = [
        model[0].predict([img_batch, r,v,c,u]),
        model[1].predict([img_batch, r]),
        model[2].predict([img_batch, v]),
        model[3].predict([img_batch, c]),
    ]
    
    for k in range(len(y_pred[0][0])):
        uu_p = y_pred[0][3][k].max(-1)
        if uu_p > unseen_th:
            rr=y_pred[0][0][k].astype('float16')
            vv=y_pred[0][1][k].astype('float16')
            cc=y_pred[0][2][k].astype('float16')       
        else:
            rr=y_pred[1][0][k].astype('float16')
            vv=y_pred[2][0][k].astype('float16')
            cc=y_pred[3][0][k].astype('float16')
        all_preds['r'].append(rr)
        all_preds['v'].append(vv)
        all_preds['c'].append(cc)

def postprocess(preds, num_classes, EXP = -1.2):
    p0 = np.argmax(preds,axis=1)

    s = pd.Series(p0)
    vc = s.value_counts().sort_index()
    df = pd.DataFrame({'a':np.arange(num_classes),'b':np.ones(num_classes)})
    df.b = df.a.map(vc)
    df.fillna(df.b.min(),inplace=True)
    mat1 = np.diag(df.b.astype('float32')**EXP)

    p0 = np.argmax(preds.dot(mat1), axis=1)
    
    return p0

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', type=int, default=123)
    parser.add_argument('--input_size', type=str, default='224,224')
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--backbone', type=str, default='efficientnet-b3')
    parser.add_argument('--weights', type=str, default='../input/efnet-b3-cosface-val9-3/best.h5')
    parser.add_argument('--r_weights', type=str, default='../input/efnet-b3-rcv-models/r_best.h5')
    parser.add_argument('--v_weights', type=str, default='../input/efnet-b3-rcv-models/v_best.h5')
    parser.add_argument('--c_weights', type=str, default='../input/efnet-b3-rcv-models/c_best.h5')
    parser.add_argument('--mode',type=str, default='all')
    args, _ = parser.parse_known_args()

    org_height = 137
    org_width = 236
    args.input_size = tuple(int(x) for x in args.input_size.split(','))
    np.random.seed(args.seed)
    tf.random.set_seed(args.seed)

    u_model = get_model(input_size=args.input_size + (3, ), backbone=args.backbone,weights=None)
    u_model.load_weights(args.weights)
    r_model = get_r_model(input_size=args.input_size + (3, ), backbone=args.backbone,weights=None)
    r_model.load_weights(args.r_weights)
    v_model = get_v_model(input_size=args.input_size + (3, ), backbone=args.backbone,weights=None)
    v_model.load_weights(args.v_weights)
    c_model = get_c_model(input_size=args.input_size + (3, ), backbone=args.backbone,weights=None)
    c_model.load_weights(args.c_weights)
    
    model = [u_model, r_model, v_model, c_model]
    
    #print(model.summary())
    
    all_preds = dict(r = [], v = [], c = [])
    all_image_ids = []
    img_batch = []
    for i in tqdm(range(4)):
        parquet_fn = f'../input/bengaliai-cv19/test_image_data_{i}.parquet'
        #parquet_fn = f'../input/bengaliai-cv19/train_image_data_{i}.parquet' # to check memory error
        all_images = pd.read_parquet(parquet_fn)
        image_ids = all_images['image_id'].values
        all_images = all_images.iloc[:, 1:].values
        for k in tqdm(range(len(image_ids))):
            all_image_ids.append(image_ids[k])
            img = all_images[k]
            img = normalize_image(img, org_width, org_height, args.input_size[1], args.input_size[0])
            img_batch.append(np.dstack([img] * 3))
            if len(img_batch) >= args.batch_size:
                predict_batch(img_batch, model, all_preds)
                img_batch = []

        # process remaining batch
        if len(img_batch) > 0:
            predict_batch(img_batch, model, all_preds)
            img_batch = []
            
        np.save(f'r_preds{i}', all_preds['r'])
        np.save(f'v_preds{i}', all_preds['v'])
        np.save(f'c_preds{i}', all_preds['c'])
        all_preds = dict(r = [], v = [], c = [])
        
        del all_images
        gc.collect()

    del u_model, r_model, v_model, c_model, model
    gc.collect()
    
    r_preds = np.concatenate([np.load(f'r_preds{i}.npy') for i in range(4)] ,axis=0)
    r_preds = postprocess(r_preds, 168, -1.2)
    #r_preds = r_preds.argmax(axis=1)
    
    v_preds = np.concatenate([np.load(f'v_preds{i}.npy') for i in range(4)] ,axis=0)
    v_preds = postprocess(v_preds, 11, -1.2)
    #v_preds = v_preds.argmax(axis=1)
    
    c_preds = np.concatenate([np.load(f'c_preds{i}.npy') for i in range(4)] ,axis=0)
    c_preds = postprocess(c_preds, 7, -0.5)
    #c_preds = c_preds.argmax(axis=1)
    
    # create submission
    row_id, target = [], []
    for iid, r, v, c in zip(all_image_ids, r_preds, v_preds, c_preds):
        row_id.append(iid + '_grapheme_root')
        target.append(r)
        row_id.append(iid + '_vowel_diacritic')
        target.append(v)
        row_id.append(iid + '_consonant_diacritic')
        target.append(c)

    sub_fn = 'submission.csv'
    sub = pd.DataFrame({'row_id': row_id, 'target': target})
    sub.to_csv(sub_fn, index=False)
    print(f'Done wrote to {sub_fn}')

main()

In [ ]:
sub = pd.read_csv('submission.csv')
sub.head(20)